In [3]:
import pandas as pd
df = pd.read_csv('fmcad_results_2023-05-16_23-03-02/master_updated_stats.csv')

In [5]:
df.columns

Index(['relative_path', 'features', 'split_depth', 'test_suite', 'Unnamed: 1',
       'time', 'stop_reasons', 'success', 'lemma_count',
       'proofs_later_filtered', 'case_split_root_count',
       'case_split_had_vacuity', 'total_allocated', 'max_allocated',
       'file_name', 'relative_path.1', 'num_terms_added', 'total_search_time',
       'total_apply_time', 'total_rebuild_time', 'total_enodes', 'num_splits',
       'should_delete', 'deleted_colored_enodes', 'colored_unions_count',
       'total_colors_size', 'black_size', 'vacuos_colors', 'split_sizes',
       'graph_memory', 'eclasses_count', 'id_count',
       'colored_equivalences_size', 'black_colored_classes_size',
       'parent_color_tracking_size', 'error', 'split_depth.1', 'features.1'],
      dtype='object')

In [14]:
configs = df['features'].unique()
errors = {}
for config in configs:
    config_df = df[df['features'] == config]
    error_df = config_df[(config_df['error'].notnull()) & ((config_df['error'] == 'Timeout Exception') | (config_df['error'].str.contains('memory allocation')))]
    errors[config] = error_df


In [16]:
for e in errors.values():
    print(e.error.unique())
    break

['Timeout Exception' 'memory allocation of 1179664 bytes failed\n'
 'memory allocation of 86016 bytes failed\n'
 'memory allocation of 112 bytes failed\n'
 'memory allocation of 5018288 bytes failed\n'
 'memory allocation of 60129542144 bytes failed\n'
 'memory allocation of 1376256 bytes failed\n'
 'memory allocation of 2688 bytes failed\n'
 'memory allocation of 233488 bytes failed\n'
 'memory allocation of 2752512 bytes failed\n'
 'memory allocation of 589840 bytes failed\n'
 'memory allocation of 2359312 bytes failed\n'
 'memory allocation of 5018728 bytes failed\n'
 'memory allocation of 448 bytes failed\n'
 'memory allocation of 5016968 bytes failed\n'
 'memory allocation of 116752 bytes failed\n']


In [20]:
failed_benchmarks = {}
for config, error_df in errors.items():
    failed_benchmarks[config] = error_df.groupby(['relative_path', 'split_depth']).groups.keys()

In [22]:
failed_benchmarks

{'split_no_cmemo_stats': dict_keys([('clam/goal8.th', 4), ('clam/goal9.th', 3), ('clam/goal9.th', 4), ('hipspec-rev-equiv/rev-equiv-goal1.th', 2), ('hipspec-rev-equiv/rev-equiv-goal1.th', 3), ('hipspec-rev-equiv/rev-equiv-goal1.th', 4), ('hipspec-rev-equiv/rev-equiv-goal2.th', 2), ('hipspec-rev-equiv/rev-equiv-goal2.th', 3), ('hipspec-rev-equiv/rev-equiv-goal2.th', 4), ('hipspec-rev-equiv/rev-equiv-goal3.th', 2), ('hipspec-rev-equiv/rev-equiv-goal3.th', 3), ('hipspec-rev-equiv/rev-equiv-goal3.th', 4), ('hipspec-rev-equiv/rev-equiv-goal4.th', 2), ('hipspec-rev-equiv/rev-equiv-goal4.th', 3), ('hipspec-rev-equiv/rev-equiv-goal4.th', 4), ('hipspec-rev-equiv/rev-equiv-goal5.th', 2), ('hipspec-rev-equiv/rev-equiv-goal5.th', 3), ('hipspec-rev-equiv/rev-equiv-goal5.th', 4), ('hipspec-rev-equiv/rev-equiv-goal6.th', 2), ('hipspec-rev-equiv/rev-equiv-goal6.th', 3), ('hipspec-rev-equiv/rev-equiv-goal6.th', 4), ('hipspec-rev-equiv/rev-equiv-goal7.th', 2), ('hipspec-rev-equiv/rev-equiv-goal7.th', 3)

In [31]:
colored_errors = set(failed_benchmarks['split_colored_stats'])
keep_errors = set(failed_benchmarks['split_clone_keep_splits_stats'])
no_cmemo_errors = set(failed_benchmarks['split_no_cmemo_stats'])
print(len(keep_errors))
intersection = colored_errors.intersection(keep_errors)
intersection2 = keep_errors.intersection(no_cmemo_errors)
intersection3 = colored_errors.intersection(no_cmemo_errors)
percentage = len(intersection) / len(colored_errors.union(keep_errors))
percentage2 = len(intersection2) / len(no_cmemo_errors.union(keep_errors))
percentage3 = len(intersection3) / len(colored_errors.union(no_cmemo_errors))
print(percentage)
print(percentage2)
print(percentage3)

116
0.6805555555555556
0.40418118466898956
0.43902439024390244


In [54]:
# find all lines in errors for intersection
intersection_errors = {}
for config, error_df in errors.items():
    print(error_df.columns)
    intersection_errors[config] = error_df[error_df.apply(lambda row: (row['relative_path'], row['split_depth']) in intersection, axis=1)]


Index(['relative_path', 'features', 'split_depth', 'test_suite', 'Unnamed: 1',
       'time', 'stop_reasons', 'success', 'lemma_count',
       'proofs_later_filtered', 'case_split_root_count',
       'case_split_had_vacuity', 'total_allocated', 'max_allocated',
       'file_name', 'relative_path.1', 'num_terms_added', 'total_search_time',
       'total_apply_time', 'total_rebuild_time', 'total_enodes', 'num_splits',
       'should_delete', 'deleted_colored_enodes', 'colored_unions_count',
       'total_colors_size', 'black_size', 'vacuos_colors', 'split_sizes',
       'graph_memory', 'eclasses_count', 'id_count',
       'colored_equivalences_size', 'black_colored_classes_size',
       'parent_color_tracking_size', 'error', 'split_depth.1', 'features.1'],
      dtype='object')
Index(['relative_path', 'features', 'split_depth', 'test_suite', 'Unnamed: 1',
       'time', 'stop_reasons', 'success', 'lemma_count',
       'proofs_later_filtered', 'case_split_root_count',
       'case_split_h

In [60]:
fixed_colored = intersection_errors['split_colored_stats'][['relative_path', 'split_depth', 'error']]
fixed_keep = intersection_errors['split_clone_keep_splits_stats'][['relative_path', 'split_depth', 'error']]

# Have colored_error and keep_error in same df when combiingin by relative_path and split_depth
# first index
fixed_colored = fixed_colored.set_index(['relative_path', 'split_depth'])
fixed_keep = fixed_keep.set_index(['relative_path', 'split_depth'])
fixed_colored = fixed_colored.rename(columns={'error': 'colored_error'})
fixed_keep = fixed_keep.rename(columns={'error': 'keep_error'})
fixed_colored['keep_error'] = fixed_keep['keep_error']
fixed_colored

,,colored_error,keep_error
relative_path,split_depth,,
hipspec-rotate/rotate-goal7.th,3,Timeout Exception,memory allocation of 4194304 bytes failed\n
leon-amortize-queue/amortize-queue-goal12.th,3,Timeout Exception,memory allocation of 682480 bytes failed\n
leon-amortize-queue/amortize-queue-goal8.th,3,Timeout Exception,memory allocation of 240 bytes failed\n
leon-amortize-queue/amortize-queue-goal5.th,3,Timeout Exception,memory allocation of 1203344 bytes failed\n
leon-amortize-queue/amortize-queue-goal7.th,3,Timeout Exception,memory allocation of 8 bytes failed\n
...,...,...,...
leon-amortize-queue/amortize-queue-goal10.th,2,Timeout Exception,memory allocation of 240 bytes failed\n
leon-amortize-queue/amortize-queue-goal6.th,2,Timeout Exception,memory allocation of 11004000 bytes failed\n
leon-amortize-queue/amortize-queue-goal4.th,2,Timeout Exception,memory allocation of 240 bytes failed\n


In [67]:
# Check fixed colored where index releative_path contains leon-amortize-queue
fixed_colored[fixed_colored.index.get_level_values('relative_path').str.contains('leon-amortize-queue')].shape

(53, 2)

In [71]:
fixed_colored[fixed_colored.index.get_level_values('relative_path').str.contains('hipspec-rotate')]

colored_error   
relative_path                  split_depth                      
hipspec-rotate/rotate-goal7.th 3            Timeout Exception  \
                               4            Timeout Exception   

                                                                             keep_error  
relative_path                  split_depth                                               
hipspec-rotate/rotate-goal7.th 3            memory allocation of 4194304 bytes failed\n  
                               4                 memory allocation of 36 bytes failed\n